# __Oxidation notebook__

### RUN THIS CELL BEFORE YOU START THE NOTEBOOK


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import numpy as np
import ipywidgets as widgets
from matplotlib.animation import FuncAnimation
from IPython.display import display, HTML, Markdown
from base64 import b64encode

# *Theory Section*

## __Thermal oxidation__
__Thermal oxidation a type of process where silicon is
transformed into its oxide at high
temperature (850 ºC < T < 1200 ºC)
using either water vapor (“wet
oxidation”) or oxygen (“dry oxidation”). They are both Chemical Vapor Deposition (CVD) techniques as the deposition happends thanks to a chemical reaction at the surface of the substrat. They are thin film deposition techniques but are in a separate notebook as they are specific processes but widely used in industry for most of the processes. The high temperature speeds up the chemical reaction. At room temperature the oxidation wouldn't be possible (or limited to a few nanometers).__

Oxidation of silicon is crucial as the oxide of silicon SiO2 is an extremely important material in semi-conductor technology. It is used for various purposes such as gate dielectric in MOS (Metal-Oxide-Semiconductor) transistors, insulator layers in integrated circuits, and protective layers against contamination and damage. The highly insulating nature of SiO2 allows it to be an effective barrier between different layers in the device, preventing current leakage and ensuring high performance. Additionally, SiO2 also possesses excellent thermal stability and good chemical resistance, making it an ideal choice for use in harsh environments. Moreover, it plays a fundamental role in photolithography processes, which are vital for pattern definition in semiconductor device fabrication. Oxidation of silicon thus has a direct impact on the performance and reliability of a wide range of electronic devices.

Both techniques are performed by introducting the wafers inside furnaces which can heat up at high temperatures while injecting gazes in a vaccum.

* __Wet oxidation__ is obtained done by adding $H_2O$ vapor inside the furnace. The $SiO_2$ is obtained by the following reaction: $ Si (s) + H_2O (l) = SiO_2 + 2H_2 (g) $
* __Dry oxidation__ is obtained done by adding $O_2$ vapor inside the furnace. The $SiO_2$ is obtained by the following reaction: $Si (s) + O_2 (g) = SiO_2 (s)$
<br>

The thickness of the layer first grows linearly (thickness ~ time) but then follows a square root law (thickness ~ $\sqrt time$ as the oxygene or water molecules need to diffuse through the $SiO_2$ to oxidize the $Si$. As you will see later the type of oxidation greatly depends on the type of $SiO_2$ layer you need.

<img src="centrotherm image.JPG" width="600" height="400">

<br>

_This the image of a typical furnace used for oxidation processes. They can perform both types of oxidation depending on the type of gas used and the gas diffuser system installed inside the furnace. As the processes are very long, they are designed for high throughput. In the zoomed image you can see the inside of the furnace where the process happends._

Oxidation also works with other materials than silicon. Also the same machine can be used to diffuse other materials than oxygen or water vapor. For instance nitrogren can be used for nitride deposition ($Si_3N_4$).

`Run the follwing cell to display the oxidation clip.`

In [ ]:
mp4 = open('oxidation_clip.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=1000 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

_Clip description:_ This small clip shows you the processing of a batch of wafer inside a thermal oxidation furnace. As the process are very long the wafer are always processed in batch. They are hold by a special holder which resist the extrem heat of the machine. These furnaces can perform both types of oxidation. However in practice each chamber of the furnace is reserved for a single type of oxidation for purity and conveniancy conserns. Moreover this step tends to always happend in the first steps of the process as it requires to have access to the silicon of the wafer (which isn't possible if it is covered by layers of materials). The heat of the process also makes it almost incompatible with any other previous step as could damage some layers or create great stress in the structure.

# *Exercice Section*

* ## Goal of the exercice

The goal of this notebook is to understand the effect of the parameters used in and oxidation process. In a first part you will import data of thickness versus time for different type of oxidation, temperature and time of process. Then you will be able to visualise the growth curves depending on the choice. 

* ## Parameters selection

Play with the following graphs to find the most optimized parameters for your task. Select the tempareture and the type of oxidation with the buttons and find the time you want for the thickness you need to deposit. 
<br>
_The file 'oxi_data.xlsx' contains real data based on measurements made by the engineers of the CMi._

In [ ]:
%matplotlib widget

file_path = 'oxi_data.xlsx'  
#this file contains the data for many processes possible
#eache colum is a process, and the data in each cell is a pair of time;thickness values
#indeed separated by a ;
df = pd.read_excel(file_path, sheet_name=0)
#df is a structure that contains a list of the column titles in df.columns

dropdown1 = widgets.Dropdown(options=list(df.columns), description='Select column: ')
#now a dropdown selector has been created with all the titles of the columns found in the data file.
target_th= 200 #nm, this is your thickness target 

#print(enumerate(df['dry_880']))

def interpolate_data(column):
    x_vals = []
    y_vals = []
    for i, val in enumerate(column):
        if pd.notnull(val):
            # Split the string into two values separated by ";"
            x, y = val.split(";")
            x_vals.append(int(x))
            y_vals.append(int(y))
    # Create an array of x values between the minimum and maximum values in x_vals
    new_x = np.arange(min(x_vals), max(x_vals) + 1)
    # Interpolate y values for the new x values
    f = interp1d(x_vals, y_vals, kind='linear')
    new_y = f(new_x)
    return new_x, new_y

# Define function to update plot based on dropdown selection
def update_plot(column1, target):
    x1, y1 = interpolate_data(df[column1])
    #you can add mo x,y variables, and the respective columns in the update plot arguments
    #if you want to plot more than one data curve
    plt.figure(figsize=(11, 6))
    plt.plot(x1, y1)
    #you can plot more data curves adding a line here
    plt.plot([0,max(x1)],[target_th,target_th]) #line for plotting the target thickness
    plt.title("Interpolated Data for {}".format(column1))
    plt.xlabel("time [min]")
    plt.ylabel("thickness [nm]")
    plt.grid(which='both', linestyle='--', alpha=0.5)
    plt.show()

# Call update_plot function when dropdown selection changes
widgets.interactive(update_plot, column1=dropdown1, target=target_th)

#you can write here code for interpolating the exact time for the curve with the target value. 
#hint: use the interp1d function, almost like above... You can start: 
##### def interpolate_time(column,target_thickness)
##
##
##
##
#target_time=interpolate_time(dropdown1,target_th)
#convert the time from minutes, to hh:mm

* ## Visualization of the process

From the previous part, you are now informed to make a choice of process and duration. Indeed during your oxidation there are no sensors informing you of the oxide thickness so you have to choose this time prior to the oxidation. The oxide thickness is only measurerable after the process through metrology. Enter below the values selected. The values should be exactly one of those available in the previous part, otherwise it will not work.
<br>
__Fill the following values and run the cell to save them.__

In [ ]:
time_of_oxidation = 3850 # in minutes
type_of_oxidation = 'dry' # wet or dry (as a string)
temperature_of_oxidation = 880 #in °C (one of the temperatures previously given) 

The following code allows you to visualize your process for the selected parameters. 
<br>
There are two things to observe. First the thickness that you obtain from what you selected, second you have a small animation which represents the oxidation process.

In [ ]:
column = type_of_oxidation + '_' + str(temperature_of_oxidation)
num_points = 50

#PARAMETERS TO TUNE 
words = column.split('_')
if words[0] == 'dry':
    min_scale = 0.97
    max_scale = 1.03
    init_Si_thickness = 1
    y_lim = 2.5
else:
    min_scale = 0.95
    max_scale = 1.05
    init_Si_thickness = 3
    y_lim = 8

    
# Convert the semicolon-separated strings into separate columns for time and thickness
df[['time', 'thickness']] = df[column].str.split(';', expand=True).astype(float)

# Filter the data based on time_of_oxidation
df_subset = df[df['time'] <= time_of_oxidation]

# Create a limited linspace with the interpolation curve and add noise to the data
noise = np.random.uniform(min_scale, max_scale, size=num_points)
x_vals, y_vals = df_subset['time'].tolist(), df_subset['thickness'].tolist()
x = np.linspace(min(x_vals), max(x_vals), num_points)
y = np.interp(x, x_vals, y_vals)
y = y * noise


fig, ax = plt.subplots(figsize=(12, 6))
# Set up the plot elements


silicon = plt.Rectangle((0, 0), 1, init_Si_thickness, color='black', label='Silicon')
oxide_upward = plt.Rectangle((0, init_Si_thickness), init_Si_thickness, 0, color='darkgrey', label='SiO2 (upward)')
oxide_downward = plt.Rectangle((0, 0), init_Si_thickness, 0, color='grey', label='SiO2 (downward)')
ax.add_patch(silicon)
ax.add_patch(oxide_upward)
ax.add_patch(oxide_downward)
# Set up the axes and labels
ax.set_xlim(0, 1)
ax.set_ylim(0, y_lim)
ax.axhline(y= init_Si_thickness, color='gray', linestyle='--', linewidth=1, label='Original Si height')
ax.legend()

    # Add title to the plot
title = ax.set_title('Deal-Grove Model')
title.set_position([.5, 1.1]) # Adjust position of the title
title.set_fontsize(10) # Adjust font size of the title

def update(frame):
    oxide_thickness = y[frame] * 1e-3  # Convert thickness from nm to µm
    downward_growth = oxide_thickness * 0.45  # 45% of oxide thickness grows downward
    upward_growth = oxide_thickness * 0.55  # 55% of oxide thickness grows upward
    silicon_height = 1 - downward_growth  # Calculate remaining silicon height
    oxide_upward.set_height(upward_growth)
    oxide_downward.set_height(downward_growth)
    oxide_downward.set_y(init_Si_thickness - downward_growth)  # Adjust downward oxide layer position
    ax.set_yticks([0, init_Si_thickness, init_Si_thickness - downward_growth, init_Si_thickness + upward_growth])
    ax.set_ylim(0,y_lim)

    ax.set_yticklabels([0, f"{init_Si_thickness} µm (Si)", f"{downward_growth:.2f} µm (down)", f"{upward_growth:.2f} µm (upward)"])
    t = x[frame] # Get the exact value of t
    title.set_text(f"Growth of the $SiO_2$ at (t = {t:.0f} min)")
    # Print final thickness value
    if frame == len(x) - 1:
        display(Markdown(f"<h2>Final oxide thickness: {oxide_thickness * 1000:.2f} nm</h2>"))
        
ani = FuncAnimation(fig, update, frames=len(x), interval=100, repeat=False)

plt.close()
# Display the animation in JupyterLab
html_code = ani.to_jshtml()
centered_html = f'<div style="display: flex; justify-content: center;">{html_code}</div>'
display(HTML(centered_html))

You can move the slider under the image to advance the time of the oxidation process. As the time advances, the oxide layer is represented with two colors:
- The darker part is the oxide that ends up located under the original surface location.
- The lighter part the oxdie that ends up above the original surface location. 
In reality there is no boundary between these two regions. The objective in using two colors is only to illustrate that the new oxide layer does not grow on top of the silicon. On the contrary, the growth of the oxide is achieved at the expense of consuming some of the Si (the one that was occupying the dark volume). The volume of the $SiO_2$ originated by the chemical reaction is greater than the one of the silicon consumed, so the $SiO_2$ layer occupies the dar and light volumes now. 

During the process the oxygen atoms are able to diffuse through the already-formed $SiO_2$ layer, reach the Si-SiO2 interface (between black and dark regions), and react with more silicon atoms to form $SiO_2$. Here part of the $Si$ atoms is consumed. As new $SiO_2$ molecules are formed at the Si-SiO2 interface, the molecular volume increase pushes the existing $SiO_2$ layer further "upwards" into the oxidizing environment. That is why part of the layer is below the original interface and part of it above. 

This bidirectional growth, due to the $SiO_2$ layer thickness being larger than the amount of silicon consumed, results in roughly half of the oxide growth extending into the silicon substrate and the other half extending outward. This observation is crucial for understanding and predicting the final thickness and location of the oxide layer, especially in the context of semiconductor device fabrication where precise control over layer dimensions is necessary.

# __This is the end of the oxidation notebook__

Mind you that you can come back to this notebook at any time and change the parameters as many times as you need to answer all the questions in the exercises. 